In [1]:
!wget http://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes -O mm10.chrom.sizes 

--2020-06-13 17:16:19--  http://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1405 (1.4K) [text/plain]
Saving to: ‘mm10.chrom.sizes’

mm10.chrom.sizes    100%[===================>]   1.37K  --.-KB/s    in 0s      

2020-06-13 17:16:19 (81.7 MB/s) - ‘mm10.chrom.sizes’ saved [1405/1405]



In [43]:
#get 1kb around summits
#!zcat Oct4/idr-optimal-set.summit.bed.gz Sox2/idr-optimal-set.summit.bed.gz Nanog/idr-optimal-set.summit.bed.gz Klf4/idr-optimal-set.summit.bed.gz | perl -lane 'print $F[0]."\t".(($F[1]+$F[9]))."\t".(($F[1]+$F[9]))' | bedtools slop -g mm10.chrom.sizes -b 500 | perl -lane 'if ($F[2]-$F[1]==1000) {print $F[0]."\t".$F[1]."\t".$F[2]."\t1"}' | sortBed | gzip -c > 1k_around_summits.bed.gz
!zcat Oct4/idr-optimal-set.summit.bed.gz | perl -lane 'print $F[0]."\t".(($F[1]+$F[9]))."\t".(($F[1]+$F[9]))' | bedtools slop -g mm10.chrom.sizes -b 500 | perl -lane 'if ($F[2]-$F[1]==1000) {print $F[0]."\t".$F[1]."\t".$F[2]."\t1"}' | sortBed | gzip -c > 1k_around_summits.bed.gz
#split into train, valid, test sets
!zcat 1k_around_summits.bed.gz | egrep -w 'chr2|chr3|chr4' | gzip -c > test_1k_around_summits.bed.gz
!zcat 1k_around_summits.bed.gz | egrep -w 'chr1|chr8|chr9' | gzip -c > valid_1k_around_summits.bed.gz
!zcat 1k_around_summits.bed.gz | egrep -w -v 'chr1|chr2|chr3|chr4|chr9' | gzip -c > train_1k_around_summits.bed.gz

In [3]:
from collections import OrderedDict
from seqdataloader.batchproducers import coordbased
from seqdataloader.batchproducers.coordbased import coordstovals
from seqdataloader.batchproducers.coordbased import coordbatchproducers
from seqdataloader.batchproducers.coordbased import coordbatchtransformers
from seqdataloader.batchproducers.coordbased.coordstovals.core import CoordsToValsJoiner
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import AbstractCountAndProfileTransformer 
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import LogCountsPlusOne
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import SmoothProfiles
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import BigWigReader 
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import smooth_profiles
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import rolling_window
from seqdataloader.batchproducers.coordbased.coordstovals.bigwig import MultiTrackCountsAndProfile
import keras
import keras.layers as kl
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_probability as tfp
import numpy as np
from seqdataloader.batchproducers.coordbased.core import Coordinates, KerasBatchGenerator, apply_mask
from keras.models import load_model
import os
from keras.utils import CustomObjectScope
from seqdataloader.batchproducers.coordbased.core import Coordinates

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [44]:
#Written by Žiga Avsec
import pyBigWig

#loss function
def multinomial_nll(true_counts, logits):
    """Compute the multinomial negative log-likelihood
    Args:
      true_counts: observed count values
      logits: predicted logit values
    """
    counts_per_example = tf.reduce_sum(true_counts, axis=-1)
    dist = tfp.distributions.Multinomial(total_count=counts_per_example,
                                         logits=logits)
    return (-tf.reduce_sum(dist.log_prob(true_counts)) / 
            tf.to_float(tf.shape(true_counts)[0]))

#Written by Žiga Avsec
class MultichannelMultinomialNLL(object):
    def __init__(self, n):
        self.__name__ = "MultichannelMultinomialNLL"
        self.n = n

    def __call__(self, true_counts, logits):
        for i in range(self.n):
            loss = multinomial_nll(true_counts[..., i], logits[..., i])
            if i == 0:
                total = loss
            else:
                total += loss
        return total

    def get_config(self):
        return {"n": self.n}
    
#poisson loss but we take the exponent of the
# labels to convert them from log(counts+1)
# to counts.
def exponentiate_label_poisson(true, preds):
    true_counts = tf.exp(true)-1
    return tf.keras.losses.poisson(y_true=true_counts,
                                   y_pred=preds)


def poisson_loglinear_loss(true, preds):
    true_counts = tf.exp(true)-1
    preds_counts = tf.exp(preds)
    return tf.keras.losses.poisson(y_true=true_counts,
                                   y_pred=preds_counts)

In [51]:
# If we want to avoid zero-padding, then the size of the output predictions
# will depend on the size of the input sequence supplied. We define the
# API for an AbstractProfileModel class which returns the length of the
# output profile in addition to returning the model, given information
# on the size of the input sequence and the model parameters.
        
class AbstractProfileModel(object):
    
    def get_output_profile_len(self):
        raise NotImplementedError()
  
    def get_model(self):
        raise NotImplementedError()

# The architecture by Žiga Avsec involves residual connections, which means
# that the layers being added together in an elementwise fashion need
# to have the same dimensions. To achieve this without zero-padding, we
# have to trim away the flanks of earlier convolutional layers. That
# is what this function is meant to do. (Note that the original BP-net
# architecture zero-pads; this is a modification to avoid the zero
# padding and use information on actual sequence instead)

def trim_flanks_of_conv_layer(conv_layer, output_len, width_to_trim, filters):
    layer = keras.layers.Lambda(
        lambda x: x[:,
          int(0.5*(width_to_trim)):-(width_to_trim-int(0.5*(width_to_trim)))],
        output_shape=(output_len, filters))(conv_layer)
    return layer


#This model architecture is based on BP-Net by Žiga Avsec
# https://drive.google.com/file/d/1kg6Ic0-FvJtVUva9Mh3FPnOAHJcN6VB-/view
#It has been modified for this specific use-case.
class BPnetArch(AbstractProfileModel):   

    def __init__(self, counts_loss,
                       log_space_preds,
                       counts_hiddenlayer_size,
                       num_counts_hidden_layers,
                       input_seq_len, c_task_weight, filters,
                       n_dil_layers, conv1_kernel_size,
                       dil_kernel_size,
                       outconv_kernel_size, lr,
                       seed=1234):
        self.counts_loss = counts_loss
        self.log_space_preds = log_space_preds
        print("LOG SPACE PREDS IS SET TO",log_space_preds)
        self.counts_hiddenlayer_size = counts_hiddenlayer_size
        self.num_counts_hidden_layers = num_counts_hidden_layers
        self.input_seq_len = input_seq_len
        self.c_task_weight = c_task_weight
        self.filters = filters
        self.n_dil_layers = n_dil_layers
        self.conv1_kernel_size = conv1_kernel_size
        self.dil_kernel_size = dil_kernel_size
        self.outconv_kernel_size = outconv_kernel_size
        self.lr = lr
        self.seed = seed
    
    def get_embedding_len(self):
        embedding_len = self.input_seq_len
        embedding_len -= (self.conv1_kernel_size-1)     
        for i in range(1, self.n_dil_layers+1):
            dilation_rate = (2**i)
            embedding_len -= dilation_rate*(self.dil_kernel_size-1)
        return embedding_len
    
    def get_output_profile_len(self):
        embedding_len = self.get_embedding_len()
        out_profile_len = embedding_len - (self.outconv_kernel_size - 1)
        return out_profile_len
    
    def get_keras_model(self):
      
        np.random.seed(self.seed)
        tf.set_random_seed(self.seed)

        out_pred_len = self.get_output_profile_len()
        
        #'inp' is the one-hot encoded DNA sequence input
        inp = kl.Input(shape=(self.input_seq_len, 4), name='sequence')
        first_conv = kl.Conv1D(filters=self.filters,
                               kernel_size=self.conv1_kernel_size,
                               padding='valid',
                               activation='relu')(inp)
        
        #Need to keep track of the layer size for trimming purposes when
        # we get to the residual connections.
        
        curr_layer_size = self.input_seq_len - (self.conv1_kernel_size-1)
        
        #Define input layers for the control tracks - both counts and profile
        #Dimension is '1' for the ChIP-seq control counts because the positive
        # and negative strands are added together
        
        patchcap_logcount = kl.Input(
            shape=(2,), name="patchcap.logcount")
        #if working with raw counts, go from logcount->count
        if (self.log_space_preds==False):
            patchcap_count_touse = kl.Lambda(lambda x: tf.exp(x)-1)(patchcap_logcount) 
        else:
            patchcap_count_touse = patchcap_logcount
        
        patchcap_profile = kl.Input(
            shape=(out_pred_len, 2), name="patchcap.profile")
        
        #Gather together all the tensors representing the model inputs
        model_inputs = [
            inp,
            patchcap_logcount,
            patchcap_profile
        ]
        
        #Prepare the stack of dilated convolutions with residual connections
        prev_layer = first_conv
        for i in range(1, self.n_dil_layers + 1):
            dilation_rate = 2**i
            conv_output = kl.Conv1D(filters=self.filters,
                                  kernel_size=self.dil_kernel_size,
                                  padding='valid',
                                  activation='relu',
                                  dilation_rate=dilation_rate)(prev_layer)          
            width_to_trim = dilation_rate*(self.dil_kernel_size-1)
            curr_layer_size = (curr_layer_size - width_to_trim)
            prev_layer = trim_flanks_of_conv_layer(
              conv_layer=prev_layer, output_len=curr_layer_size,
              width_to_trim=width_to_trim, filters=self.filters)
            prev_layer = kl.merge.Add()([prev_layer, conv_output])

        combined_conv = prev_layer

        #gap = GlobalAveragePooling. This layer is used as input for the
        # counts prediction tasks.
        gap_combined_conv = kl.GlobalAvgPool1D()(combined_conv)
        
        lossarr = []
        lossweightsarr = []
        model_outputs = []
    
        #Define the output layers for the counts prediction tasks
        for countouttaskname, numunits, countcontrolinp in [
                    #("CHIPNexus.NANOG.logcount", 2, patchcap_count_touse),
                    ("CHIPNexus.OCT4.logcount", 2, patchcap_count_touse),
                    #("CHIPNexus.KLF4.logcount", 2, patchcap_count_touse),
                    #("CHIPNexus.SOX2.logcount", 2, patchcap_count_touse)
                ]:
            precount_out = kl.concatenate([gap_combined_conv,
                                           countcontrolinp], axis=-1)
            for i in range(self.num_counts_hidden_layers):
                precount_out = kl.Dense(
                    units=self.counts_hiddenlayer_size,
                    activation="relu")(precount_out)
            count_out = kl.Dense(
                units=numunits,
                name=countouttaskname,
                activation=("linear" if self.log_space_preds else "softplus"))(
                    precount_out)
            model_outputs.append(count_out)
            lossarr.append(self.counts_loss)
            lossweightsarr.append(self.c_task_weight)

        #Define the output layers for the profile prediction tasks
        for profileouttaskname, numunits, profilecontrolinp in [
            #("CHIPNexus.NANOG.profile", 2, patchcap_profile),
            ("CHIPNexus.OCT4.profile", 2, patchcap_profile),
            #("CHIPNexus.KLF4.profile", 2, patchcap_profile),
            #("CHIPNexus.SOX2.profile", 2, patchcap_profile)
        ]:
                                                                
        
            profile_out_precontrol = kl.Conv1D(
                                      filters=numunits,
                                      kernel_size=self.outconv_kernel_size,
                                      padding='valid')(combined_conv)
            profile_out = kl.Conv1D(filters=numunits, kernel_size=1, name=profileouttaskname)(kl.concatenate([profile_out_precontrol,profilecontrolinp], axis=-1))

            model_outputs.append(profile_out)
            lossarr.append(MultichannelMultinomialNLL(numunits))
            lossweightsarr.append(1.0)

        #Compile the model and return it
        model = keras.models.Model(inputs=model_inputs, outputs=model_outputs)
        model.compile(keras.optimizers.Adam(lr=self.lr),
                      loss=lossarr,
                      loss_weights=lossweightsarr)
        return model

seq_len = 1346
out_pred_len = 1000
#out_pred_len = modelwrapper.get_output_profile_len()
#print(out_pred_len, seq_len-out_pred_len)   
    
    
# the code below is used to prepare instances of keras.utils.Sequence that
# can be supplied to model.fit_generator(...)
# Note that we log-transform our counts using np.log(counts+1)
# Also note that the profiles for the control are smoothed by windows of
# size 1 and 50 (smoothing by a window of size 1 just returns
# the original profile)

inputs_coordstovals = coordstovals.core.CoordsToValsJoiner(
    coordstovals_list=[
      coordbased.coordstovals.fasta.PyfaidxCoordsToVals(
        genome_fasta_path="mm10_no_alt_analysis_set_ENCODE.fasta",
        mode_name="sequence",
        center_size_to_use=seq_len),
      coordstovals.bigwig.PosAndNegSeparateLogCounts(
        counts_mode_name="patchcap.logcount",
        profile_mode_name="patchcap.profile",
        pos_strand_bigwig_path="patchcap/counts.pos.bw",
        neg_strand_bigwig_path="patchcap/counts.neg.bw",
        center_size_to_use=out_pred_len),
    ]
)
    

targets_coordstovals = CoordsToValsJoiner(
    coordstovals_list=[
      #coordstovals.bigwig.PosAndNegSeparateLogCounts(
      #  counts_mode_name="CHIPNexus.NANOG.logcount",
      #  profile_mode_name="CHIPNexus.NANOG.profile",
      #  pos_strand_bigwig_path="Nanog/counts.pos.bw",
      #  neg_strand_bigwig_path="Nanog/counts.neg.bw",
      #  center_size_to_use=out_pred_len),
      coordstovals.bigwig.PosAndNegSeparateLogCounts(
        counts_mode_name="CHIPNexus.OCT4.logcount",
        profile_mode_name="CHIPNexus.OCT4.profile",
        pos_strand_bigwig_path="Oct4/counts.pos.bw",
        neg_strand_bigwig_path="Oct4/counts.neg.bw",
        center_size_to_use=out_pred_len),
      #coordstovals.bigwig.PosAndNegSeparateLogCounts(
      #  counts_mode_name="CHIPNexus.KLF4.logcount",
      #  profile_mode_name="CHIPNexus.KLF4.profile",
      #  pos_strand_bigwig_path="Klf4/counts.pos.bw",
      #  neg_strand_bigwig_path="Klf4/counts.neg.bw",
      #  center_size_to_use=out_pred_len),"""
      #coordstovals.bigwig.PosAndNegSeparateLogCounts(
      #  counts_mode_name="CHIPNexus.SOX2.logcount",
      #  profile_mode_name="CHIPNexus.SOX2.profile",
      #  pos_strand_bigwig_path="Sox2/counts.pos.bw",
      #  neg_strand_bigwig_path="Sox2/counts.neg.bw",
      #  center_size_to_use=out_pred_len),
    ]
)

keras_train_batch_generator = KerasBatchGenerator(
  coordsbatch_producer=coordbatchproducers.SimpleCoordsBatchProducer(
      bed_file="train_1k_around_summits.bed.gz",
      batch_size=64,
      shuffle_before_epoch=True, 
      seed=1234),
  inputs_coordstovals=inputs_coordstovals,
  targets_coordstovals=targets_coordstovals,
  coordsbatch_transformer=
          coordbatchtransformers.ReverseComplementAugmenter().chain(
          coordbatchtransformers.UniformJitter(
              maxshift=200, chromsizes_file="mm10.chrom.sizes")),
)

keras_valid_batch_generator = KerasBatchGenerator(
  coordsbatch_producer=coordbatchproducers.SimpleCoordsBatchProducer(
            bed_file="valid_1k_around_summits.bed.gz",
            batch_size=64,
            shuffle_before_epoch=False, 
            seed=1234),
  inputs_coordstovals=inputs_coordstovals,
  targets_coordstovals=targets_coordstovals
)

#As a sanity check, print out the dimensions of everything in individual batches
sampinputs,samptargets = keras_train_batch_generator[0]
# for key in sampinputs:
#   print(key, sampinputs[key].shape)
for key in samptargets:
    print(key, samptargets[key].shape)

Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
CHIPNexus.OCT4.logcount (128, 2)
CHIPNexus.OCT4.profile (128, 1000, 2)


In [52]:
def extend_generator(generator):
    samp_inputs, samp_targets = generator[0]
    concat_inputs = OrderedDict([(key, []) for key in samp_inputs.keys()])
    concat_targets = OrderedDict([(key, []) for key in samp_targets.keys()])

    for batch_idx in range(len(generator)):
        batch_inputs, batch_targets = generator[batch_idx]
        for key in batch_inputs:
            concat_inputs[key].extend(batch_inputs[key])
        for key in batch_targets:
            concat_targets[key].extend(batch_targets[key])
    
    for key in concat_inputs:
        concat_inputs[key] = np.array(concat_inputs[key])
    
    for key in concat_targets:
        concat_targets[key] = np.array(concat_targets[key])
    
    return (concat_inputs, concat_targets)

from scipy.stats import spearmanr
from matplotlib import pyplot as plt

def plot_scatter(preds, targets, keys, takelogofpreds):
    fig, ax = plt.subplots(nrows=1, ncols=len(keys),
                           figsize=(5*len(keys),5))
    for i,key in enumerate(keys):
        if takelogofpreds:
            x = np.sum(np.log(preds[key]+1), axis=-1)
        else:
            x = np.sum(preds[key], axis=-1)
        y = np.sum(targets[key], axis=-1)
        ax[i].scatter(x, y, alpha=0.01)        
        ax[i].set_xlabel("Preds: "+key)
        ax[i].set_ylabel("Targets: "+key)
        ax[i].plot(x, x, color='black')
        print(key,spearmanr(x,y))
    plt.show()

In [53]:
#get the unrolled validation set
valid_concat_inputs, valid_concat_targets = extend_generator(keras_valid_batch_generator)

In [54]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

for i in range(5):
    for c_task_weight in [0,125,250,500]:
        counts_loss, log_space_preds, c_task_weight = ("mse", True, c_task_weight)
        #counts_loss, log_space_preds, c_task_weight = (poisson_loglinear_loss, True, 1)
        seed = 1234 + i*100
        print("Seed:",seed)
        modelwrapper = BPnetArch(
            counts_loss=counts_loss,
            log_space_preds=log_space_preds,
            input_seq_len=seq_len,
            counts_hiddenlayer_size=None,
            num_counts_hidden_layers=0,
            c_task_weight=c_task_weight,
            filters=64, n_dil_layers=6,
            conv1_kernel_size=21,
            dil_kernel_size=3,
            outconv_kernel_size=75,
            lr=0.001,
            seed=1234*i)

        """class PlotScatter(keras.callbacks.Callback):

            def __init__(self, takelogofpreds):
                self.takelogofpreds = takelogofpreds

            def on_epoch_begin(self, epoch, logs=None):
                print("Scatterplots before epoch",epoch)
                valid_preds = dict(zip(model.output_names, model.predict(valid_concat_inputs)))
                plot_scatter(preds=valid_preds, targets=valid_concat_targets,
                             takelogofpreds=self.takelogofpreds,
                             keys=['CHIPNexus.OCT4.logcount', 'CHIPNexus.SOX2.logcount',
                                   'CHIPNexus.NANOG.logcount', 'CHIPNexus.KLF4.logcount'])"""

        modelname = "Oct4only_ctaskweight"+str(c_task_weight)+"_seed"+str(seed)+".h5"
        print(modelname)
        model = modelwrapper.get_keras_model()
        print(model.summary())
        early_stopping_callback = keras.callbacks.EarlyStopping(
                                    patience=10,
                                    restore_best_weights=True)
        loss_history = model.fit_generator(
                            keras_train_batch_generator,
                            epochs=200,
                            validation_data=keras_valid_batch_generator,
                            callbacks=[early_stopping_callback,
                                       #PlotScatter(takelogofpreds=(log_space_preds==False))
                                      ],
                            workers=1)
        best_epoch = np.argmin(loss_history.history['val_loss'])
        for key in loss_history.history.keys():
            print(key,loss_history.history[key][best_epoch])
        model.set_weights(early_stopping_callback.best_weights)
        model.save(modelname)

Seed: 1234
LOG SPACE PREDS IS SET TO True
Oct4only_ctaskweight0_seed1234.h5
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence (InputLayer)           (None, 1346, 4)      0                                            
__________________________________________________________________________________________________
conv1d_94 (Conv1D)              (None, 1326, 64)     5440        sequence[0][0]                   
__________________________________________________________________________________________________
lambda_61 (Lambda)              (None, 1322, 64)     0           conv1d_94[0][0]                  
__________________________________________________________________________________________________
conv1d_95 (Conv1D)              (None, 1322, 64)     12352       conv1d_94[0][0]                  
________________

276/276 [==============================] - 88s 319ms/step - loss: 1568.0944 - CHIPNexus.OCT4.logcount_loss: 7.0962 - CHIPNexus.OCT4.profile_loss: 1568.0935 - val_loss: 1852.7327 - val_CHIPNexus.OCT4.logcount_loss: 8.2385 - val_CHIPNexus.OCT4.profile_loss: 1232.0364
Epoch 6/200
276/276 [==============================] - 94s 339ms/step - loss: 1545.7920 - CHIPNexus.OCT4.logcount_loss: 10.8783 - CHIPNexus.OCT4.profile_loss: 1545.7914 - val_loss: 1892.7759 - val_CHIPNexus.OCT4.logcount_loss: 13.6696 - val_CHIPNexus.OCT4.profile_loss: 1225.7758
Epoch 7/200
276/276 [==============================] - 104s 376ms/step - loss: 1524.1965 - CHIPNexus.OCT4.logcount_loss: 8.0387 - CHIPNexus.OCT4.profile_loss: 1524.1963 - val_loss: 1844.6890 - val_CHIPNexus.OCT4.logcount_loss: 8.1427 - val_CHIPNexus.OCT4.profile_loss: 1216.1868
Epoch 8/200
276/276 [==============================] - 89s 321ms/step - loss: 1503.6295 - CHIPNexus.OCT4.logcount_loss: 8.4650 - CHIPNexus.OCT4.profile_loss: 1503.6290 - val_l

Epoch 1/200
276/276 [==============================] - 80s 290ms/step - loss: 1735.7508 - CHIPNexus.OCT4.logcount_loss: 0.3845 - CHIPNexus.OCT4.profile_loss: 1687.6826 - val_loss: 2102.0823 - val_CHIPNexus.OCT4.logcount_loss: 0.2913 - val_CHIPNexus.OCT4.profile_loss: 1286.1918
Epoch 2/200
276/276 [==============================] - 86s 312ms/step - loss: 1655.9860 - CHIPNexus.OCT4.logcount_loss: 0.3092 - CHIPNexus.OCT4.profile_loss: 1617.3412 - val_loss: 2103.2781 - val_CHIPNexus.OCT4.logcount_loss: 0.2841 - val_CHIPNexus.OCT4.profile_loss: 1259.6021
Epoch 3/200
276/276 [==============================] - 82s 298ms/step - loss: 1614.2273 - CHIPNexus.OCT4.logcount_loss: 0.3108 - CHIPNexus.OCT4.profile_loss: 1575.3774 - val_loss: 2085.8052 - val_CHIPNexus.OCT4.logcount_loss: 0.2792 - val_CHIPNexus.OCT4.profile_loss: 1239.8694
Epoch 4/200
276/276 [==============================] - 81s 294ms/step - loss: 1582.0063 - CHIPNexus.OCT4.logcount_loss: 0.2935 - CHIPNexus.OCT4.profile_loss: 1545.314

/users/avanti/anaconda3/envs/py376/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.118877). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


276/276 [==============================] - 89s 324ms/step - loss: 1390.4576 - CHIPNexus.OCT4.logcount_loss: 0.2989 - CHIPNexus.OCT4.profile_loss: 1353.0898 - val_loss: 2012.8521 - val_CHIPNexus.OCT4.logcount_loss: 0.2685 - val_CHIPNexus.OCT4.profile_loss: 1173.1462
Epoch 15/200
  3/276 [..............................] - ETA: 36s - loss: 1074.1100 - CHIPNexus.OCT4.logcount_loss: 0.2613 - CHIPNexus.OCT4.profile_loss: 1041.4535

...

276/276 [==============================] - 81s 292ms/step - loss: 1058.7881 - CHIPNexus.OCT4.logcount_loss: 0.2478 - CHIPNexus.OCT4.profile_loss: 1027.8098 - val_loss: 1939.5148 - val_CHIPNexus.OCT4.logcount_loss: 0.2496 - val_CHIPNexus.OCT4.profile_loss: 1070.3367
Epoch 37/200
276/276 [==============================] - 82s 297ms/step - loss: 1056.9933 - CHIPNexus.OCT4.logcount_loss: 0.2423 - CHIPNexus.OCT4.profile_loss: 1026.7018 - val_loss: 1950.1565 - val_CHIPNexus.OCT4.logcount_loss: 0.2549 - val_CHIPNexus.OCT4.profile_loss: 1071.4789
Epoch 38/200
276/276 [==============================] - 85s 308ms/step - loss: 1068.5817 - CHIPNexus.OCT4.logcount_loss: 0.2387 - CHIPNexus.OCT4.profile_loss: 1038.7504 - val_loss: 1897.5316 - val_CHIPNexus.OCT4.logcount_loss: 0.2780 - val_CHIPNexus.OCT4.profile_loss: 1071.7305
Epoch 39/200
276/276 [==============================] - 80s 291ms/step - loss: 1055.9207 - CHIPNexus.OCT4.logcount_loss: 0.2375 - CHIPNexus.OCT4.profile_loss: 1026.2302 - val_l

Epoch 1/200
276/276 [==============================] - 82s 298ms/step - loss: 1619.4161 - CHIPNexus.OCT4.logcount_loss: 0.5995 - CHIPNexus.OCT4.profile_loss: 1469.5376 - val_loss: 2350.9490 - val_CHIPNexus.OCT4.logcount_loss: 0.3083 - val_CHIPNexus.OCT4.profile_loss: 1317.4852
Epoch 2/200
276/276 [==============================] - 80s 290ms/step - loss: 1497.3391 - CHIPNexus.OCT4.logcount_loss: 0.3056 - CHIPNexus.OCT4.profile_loss: 1420.9456 - val_loss: 2390.9666 - val_CHIPNexus.OCT4.logcount_loss: 0.3175 - val_CHIPNexus.OCT4.profile_loss: 1288.4974
Epoch 3/200
276/276 [==============================] - 81s 293ms/step - loss: 1455.6223 - CHIPNexus.OCT4.logcount_loss: 0.3094 - CHIPNexus.OCT4.profile_loss: 1378.2698 - val_loss: 2266.1763 - val_CHIPNexus.OCT4.logcount_loss: 0.2954 - val_CHIPNexus.OCT4.profile_loss: 1258.3837
Epoch 4/200
276/276 [==============================] - 82s 297ms/step - loss: 1423.6196 - CHIPNexus.OCT4.logcount_loss: 0.3038 - CHIPNexus.OCT4.profile_loss: 1347.675

Epoch 1/200
276/276 [==============================] - 79s 286ms/step - loss: 1747.5200 - CHIPNexus.OCT4.logcount_loss: 0.5541 - CHIPNexus.OCT4.profile_loss: 1470.4456 - val_loss: 2622.7126 - val_CHIPNexus.OCT4.logcount_loss: 0.3084 - val_CHIPNexus.OCT4.profile_loss: 1335.1970
Epoch 2/200
276/276 [==============================] - 79s 288ms/step - loss: 1581.5524 - CHIPNexus.OCT4.logcount_loss: 0.3105 - CHIPNexus.OCT4.profile_loss: 1426.3162 - val_loss: 2628.6506 - val_CHIPNexus.OCT4.logcount_loss: 0.2983 - val_CHIPNexus.OCT4.profile_loss: 1304.7902
Epoch 3/200
276/276 [==============================] - 81s 295ms/step - loss: 1549.8091 - CHIPNexus.OCT4.logcount_loss: 0.3105 - CHIPNexus.OCT4.profile_loss: 1394.5746 - val_loss: 2596.1970 - val_CHIPNexus.OCT4.logcount_loss: 0.2923 - val_CHIPNexus.OCT4.profile_loss: 1277.9990
Epoch 4/200
276/276 [==============================] - 82s 298ms/step - loss: 1519.0027 - CHIPNexus.OCT4.logcount_loss: 0.3150 - CHIPNexus.OCT4.profile_loss: 1361.499

/users/avanti/anaconda3/envs/py376/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.116986). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


276/276 [==============================] - 81s 295ms/step - loss: 1276.0990 - CHIPNexus.OCT4.logcount_loss: 0.2411 - CHIPNexus.OCT4.profile_loss: 1155.5459 - val_loss: 2171.9546 - val_CHIPNexus.OCT4.logcount_loss: 0.2406 - val_CHIPNexus.OCT4.profile_loss: 1135.5243
Epoch 17/200
276/276 [==============================] - 85s 307ms/step - loss: 1264.5469 - CHIPNexus.OCT4.logcount_loss: 0.2444 - CHIPNexus.OCT4.profile_loss: 1142.3461 - val_loss: 2188.0686 - val_CHIPNexus.OCT4.logcount_loss: 0.2389 - val_CHIPNexus.OCT4.profile_loss: 1142.1041
Epoch 18/200
276/276 [==============================] - 83s 300ms/step - loss: 1256.0233 - CHIPNexus.OCT4.logcount_loss: 0.2478 - CHIPNexus.OCT4.profile_loss: 1132.1346 - val_loss: 2250.9917 - val_CHIPNexus.OCT4.logcount_loss: 0.2507 - val_CHIPNexus.OCT4.profile_loss: 1120.3491
Epoch 19/200
276/276 [==============================] - 83s 300ms/step - loss: 1265.0334 - CHIPNexus.OCT4.logcount_loss: 0.2638 - CHIPNexus.OCT4.profile_loss: 1133.1294 - val_l